In [2]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel
import torch
import pandas as pd
import keras

c:\project\chatbot_dl\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.8.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT

In [4]:
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  125164032 
 er)                                                             
                                                                 
Total params: 125164032 (477.46 MB)
Trainable params: 125164032 (477.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
tokenizer.vocab_size

51200

In [6]:
data1 = pd.read_csv('data/data1.csv')
data1.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [7]:
data2 = pd.read_csv('data/data2.csv')
data2.head()

,req,res
0,너 좋아하는 차 종류 있어?,무슨 차? 자동차? 마시는 차?
1,ㅋㅋ 마시는 차 말한 거야!,"아하 나 둥글레, 옥수수, 보리차 좋아해"
2,완전 곡물류 좋아하네 ㅋㅋ,야쓰 끓이기 귀찮아서 냉침해 먹어
3,그럼 오래 걸리지 않아?,끓이는 것보다는 훨씬 오래 걸리지 ㅠ
4,근데 냉침 하는 것도 귀찮겠다 ㅜㅠ,응! 그래서 매일은 안 먹고 가끔 마셔


In [18]:
chat_data = list(zip(data1['Q'].to_list(),data1['A'].to_list()))
chat_data2 = list(zip(data2['req'].to_list(),data2['res'].to_list()))
chat_data.extend(chat_data2)
len(chat_data)

100797

In [19]:
def get_chat_data():
    bos_token = tokenizer.bos_token
    eos_token = tokenizer.eos_token
    for question, answer in chat_data:
        sent = f'{bos_token}<usr>{question}<sys>{answer}{eos_token}'
        yield tokenizer.encode(sent)

In [20]:
gen = get_chat_data()
next(gen)

[0, 2, 9349, 7888, 739, 7318, 376, 4, 12557, 6824, 9108, 9028, 7098, 25856, 1]

In [21]:
batch_size = 32
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

dataset = dataset.padded_batch(batch_size=batch_size,padded_shapes=(None,),padding_values=tokenizer.pad_token_id)

In [ ]:
adam = keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-8)
steps = len(chat_data) // batch_size
epochs = 1
for epoch in range(epochs):
    train_loss = 0

    for batch in dataset:
        with tf.GradientTape() as tape:
            result = model(batch, labels=batch)
            batch_loss = tf.reduce_mean(result[0])
        train_loss += batch_loss
    train_loss /= steps
    print(f'epoch:{epoch+1}, loss={train_loss:.4f}')

In [22]:
model.save_weight('chatbot.weight.h5')

AttributeError: 'TFGPT2LMHeadModel' object has no attribute 'save_weight'

In [23]:
text = '<s><usr>밥은 먹고 다니니?<sys>'
input_ids = tokenizer.encode(text)
input_ids

[0, 2, 14821, 8135, 15495, 14841, 7172, 406, 4]

In [24]:
input_ids = tf.convert_to_tensor([input_ids])
outputs = model.generate(input_ids, max_length=100, do_sample=True, top_k=20)
tokenizer.decode(outputs[0].numpy().tolist())

'<s><usr> 밥은 먹고 다니니?<sys>"은 이렇게 물었다.\n"그래요." 밥이 대답하자, 그건 뭔가 이상했다.\n"아, 그런 일은 어디 있니?" 밥이 말했다.\n밥은 대답 없이 고개를 끄덕였다.\n그런데 그 말은 바로 자기 자신의 마음이었다.\n밥은 대답 없이 말했다.\n"그게 뭔지 모르겠군요." 밥이 대답했다.\n"아니, 무슨 일이지?"\n밥이 대답하자, 밥은 이렇게 대답했다.\n"이것 역시 내 마음이야."\n'

In [25]:
def chatbot(user_text):
    input_ids = tokenizer.encode(f'<s><usr>{user_text}<sys>')
    input_ids = tf.convert_to_tensor([input_ids])
    outputs = model.generate(input_ids, max_length=100, do_sample=True, top_k=100, top_p=0.9)
    sentence = tokenizer.decode(outputs[0].numpy().tolist())
    return sentence.split('<sys>')[1]

In [27]:
chatbot('오늘 저녁 메뉴 추천해줘')

'에요?\n다음으로 캬~ 캬~! 예~ 자 이거 두 번째 이야기부터 해주셔야돼요?!\n와~ 캬~ 캬~ 캬~ 캬~ 캬~ 와~ 와~ 와~ 자 여기 보니까 캬~ 캬~ 진짜 대박 입었어요?!\n와~ 와~ 캬~ 캬~ 캬~ 음~ 캬~ 다들 먹고'